In [1]:
import boto3
import pandas as pd
import numpy as np

In [2]:
def download_file_from_s3(bucket, object_name, local_file_name):
    """
    Download a file from S3 to the local file system.

    :param bucket: Name of the S3 bucket
    :param object_name: S3 object name
    :param local_file_name: Local file name to save the downloaded file
    """
    # # When using IAM roles, boto3 retrieves credentials from the instance metadata
    # s3_client = boto3.client('s3')

    #When setting up credentials locally, use the following code
    session = boto3.Session()
    s3_client = session.client('s3')

    s3_client.download_file(bucket, object_name, local_file_name)

bucket_name = 'myukdata'
s3_file_name = 'Original/Member/Member_CommitteeMemberships.csv'
local_file = 'Member_CommitteeMemberships.csv'

download_file_from_s3(bucket_name, s3_file_name, local_file)

In [3]:
df = pd.read_csv('Member_CommitteeMemberships.csv')
df.info()
pd.set_option('display.max_rows', None)
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12703 entries, 0 to 12702
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   memberID            12703 non-null  int64  
 1   house               12687 non-null  float64
 2   name                12703 non-null  object 
 3   id                  12703 non-null  int64  
 4   startDate           12703 non-null  object 
 5   endDate             11241 non-null  object 
 6   additionalInfo      639 non-null    object 
 7   additionalInfoLink  0 non-null      float64
dtypes: float64(2), int64(2), object(4)
memory usage: 794.1+ KB


,memberID,house,name,id,startDate,endDate,additionalInfo,additionalInfoLink
0,1,1.0,Consolidation etc. Bills (Joint Committee),18,1997-07-28T00:00:00,2001-05-11T00:00:00,NaN,NaN
1,1,1.0,European Scrutiny Committee,69,1999-12-14T00:00:00,2001-05-11T00:00:00,NaN,NaN
2,1,1.0,"Environment, Food and Rural Affairs Committee",52,2001-07-16T00:00:00,2010-05-06T00:00:00,NaN,NaN
3,1,1.0,East of England Regional Select Committee,40,2009-03-03T00:00:00,2010-05-06T00:00:00,NaN,NaN
4,2,1.0,Public Administration Committee,128,2011-01-17T00:00:00,2015-03-30T00:00:00,NaN,NaN


In [4]:
df.drop(columns=['additionalInfo',
                 'additionalInfoLink',
                 'house',
                 'id'], inplace=True)

df.rename(columns={'memberID': 'memberId',
                     'name': 'committeeName'}, inplace=True)
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12703 entries, 0 to 12702
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   memberId       12703 non-null  int64         
 1   committeeName  12703 non-null  object        
 2   startDate      12703 non-null  datetime64[ns]
 3   endDate        11241 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 397.1+ KB


In [6]:
#Save the file to S3
import boto3
import io

def upload_df_to_s3(df, bucket, object_name):
    """
    Upload a DataFrame to an S3 bucket as CSV.

    :param df: DataFrame to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name
    :return: True if the DataFrame was uploaded, else False
    """
    # Create a buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Move to the start of the buffer
    csv_buffer.seek(0)

    # Upload the buffer content to S3
    s3_client = boto3.client('s3')
    try:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=csv_buffer.getvalue())
    except ClientError as e:
        logging.error(e)
        return False
    return True

bucket_name = 'myukdata'
object_name = 'Preprocessing/Member/MemberCommitteeMemberships.csv'
upload_df_to_s3(df, bucket_name, object_name)


True